In [7]:
import ctypes
from pathlib import Path
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import math
import pprint
from datetime import datetime

In [8]:
class MapsForgeHeader:
    magic_byte: str
    header_size: int
    file_version: int
    file_size: int
    date_of_creation: datetime
    minLat: int
    minLon: int
    maxLat: int
    maxLon: int
    tile_size: int
    projection: str
    flags: int
    mapStartLat: int
    mapStartLon: int
    start_zoom_level: int
    language:str
    comment:str
    created_by:str
    poiTags:list
    wayTags:list
    zoom_interval_configs:list


class ParsedFileName:
    country: str
    state: str
    creationDate: str
    minLong: int
    minLat: int
    sizeNorthSouth: int
    sizeWestEast: int

    def __init__(self, filename):
        self.country = filename[:2]
        self.state = filename[2:6]
        self.creationDate = filename[6:12]
        self.field1 = int(filename[12:15], 36)
        self.field2 = int(filename[15:18], 36)
        self.z1 = int(filename[18:21], 36)
        self.z2 = int(filename[21:24], 36)


def parseVBEU(data: bytes):
    idx = 0
    value = 0
    while data[idx] & 0x80:
        value += (data[idx] & 0x7F) << (7 * idx)
        idx += 1
    value += (data[idx] & 0x7F) << (7 * idx)
    idx += 1

    return value, idx


def parseVBES(data: bytes):
    idx = 0
    value = 0
    while data[idx] & 0x80:
        value += (data[idx] & 0x7F) << (7 * idx)
        idx += 1
    value += (data[idx] & 0x3F) << (7 * idx)
    value *= -1 if data[idx] & 0x40 else 1
    idx += 1

    return value, idx


def parseHeader(file: Path) -> MapsForgeHeader:
    data = open(file, "rb").read(2000)
    header = MapsForgeHeader()
    idx = 0
    header.magic_byte = data[idx : idx + 20]
    idx += 20
    header.header_size = int.from_bytes(data[idx : idx + 4])
    idx += 4


    assert header.header_size < 1900, "didn't read enough bytes"

    header.file_version = int.from_bytes(data[idx : idx + 4])
    idx += 4
    header.file_size = int.from_bytes(data[idx : idx + 8])
    idx += 8
    header.date_of_creation = datetime.fromtimestamp(
        int.from_bytes(data[idx : idx + 8]) / 1000
    )
    idx += 8
    header.minLat = int.from_bytes(data[idx : idx + 4], signed=True) / 10**6
    idx += 4
    header.minLon = int.from_bytes(data[idx : idx + 4], signed=True) / 10**6
    idx += 4
    header.maxLat = int.from_bytes(data[idx : idx + 4], signed=True) / 10**6
    idx += 4
    header.maxLon = int.from_bytes(data[idx : idx + 4], signed=True) / 10**6
    idx += 4
    header.tile_size = int.from_bytes(data[idx : idx + 2])
    idx += 2

    strlen, used_bytes = parseVBEU(data[idx:])
    idx += used_bytes
    header.projection = data[idx : idx + strlen]
    idx += strlen

    flags = data[idx]
    header.flags = flags
    idx += 1

    if flags & 0x40:
        header.mapStartLat = int.from_bytes(data[idx : idx + 4], signed=True) / 10**6
        idx += 4
        header.mapStartLon = int.from_bytes(data[idx : idx + 4], signed=True) / 10**6
        idx += 4

    if flags & 0x20:
        header.start_zoom_level = int.from_bytes(data[idx : idx + 1])
        idx += 1

    if flags & 0x10:
        strlen, used_bytes = parseVBEU(data[idx:])
        idx += used_bytes
        header.language = data[idx : idx + strlen]
        idx += strlen

    if flags & 0x08:
        strlen, used_bytes = parseVBEU(data[idx:])
        idx += used_bytes
        header.comment = data[idx : idx + strlen]
        idx += strlen

    if flags & 0x04:
        strlen, used_bytes = parseVBEU(data[idx:])
        idx += used_bytes
        header.created_by = data[idx : idx + strlen]
        idx += strlen

    if flags & 0x03:
        print("parse error, future usage fileds")

    poiTagsCnt = int.from_bytes(data[idx : idx + 2])
    idx += 2
    header.poiTags = []
    for i in range(poiTagsCnt):
        strlen, used_bytes = parseVBEU(data[idx:])
        idx += used_bytes
        tag = data[idx : idx + strlen]
        idx += strlen
        header.poiTags.append(tag)
    header.poiTags.sort()

    wayTagsCnt = int.from_bytes(data[idx : idx + 2])
    idx += 2
    header.wayTags = []
    for i in range(wayTagsCnt):
        strlen, used_bytes = parseVBEU(data[idx:])
        idx += used_bytes
        tag = data[idx : idx + strlen]
        idx += strlen
        header.wayTags.append(tag)
    header.wayTags.sort()

    amount_zoom_intervals = int.from_bytes(data[idx:idx+1])
    idx += 1

    header.zoom_interval_configs = []
    for i in range(amount_zoom_intervals):
        basezoom = int.from_bytes(data[idx : idx + 1])
        idx += 1
        minzoom = int.from_bytes(data[idx : idx + 1])
        idx += 1
        maxzoom = int.from_bytes(data[idx : idx + 1])
        idx += 1
        absstart = int.from_bytes(data[idx : idx + 8])
        idx += 8
        sizesub = int.from_bytes(data[idx : idx + 8])
        idx += 8
        header.zoom_interval_configs.append({
            "basezoom": basezoom,
            "minzoom": minzoom,
            "maxzoom": maxzoom,
            "absstart": absstart,
            "sizesub": sizesub,
        })

    # assert(idx - 24 == header.header_size)

    return header


def parseFilename(filename) -> ParsedFileName:
    return ParsedFileName(filename)


In [9]:
files = [f for f in Path("./").glob("*.map") if f.is_file()]
files = [f for f in Path("./my_maps").rglob("*.map") if f.is_file()]
files = [f for f in Path("./factory_data/00D6-5BD8/iGPSPORT/").rglob("*.map") if f.is_file()]

files =  sorted(files, key=lambda x: x.name)
data = []


print("--- files ----   (", len(files), ")")

for i in files:
    print(i)

--- files ----   ( 432 )
factory_data/00D6-5BD8/iGPSPORT/Maps/AR010023031021O3S204B068.map
factory_data/00D6-5BD8/iGPSPORT/Maps/AR020023031024S3T1004006.map
factory_data/00D6-5BD8/iGPSPORT/Maps/AR03002303101Y33M702R03K.map
factory_data/00D6-5BD8/iGPSPORT/Maps/AR040023031021O3LG03702U.map
factory_data/00D6-5BD8/iGPSPORT/Maps/AR05002303101W43Z205N03K.map
factory_data/00D6-5BD8/iGPSPORT/Maps/AR06002303102063PA02K046.map
factory_data/00D6-5BD8/iGPSPORT/Maps/AR07002303102413NO02L02J.map
factory_data/00D6-5BD8/iGPSPORT/Maps/AR080023031023C3PR01X02Y.map
factory_data/00D6-5BD8/iGPSPORT/Maps/AR090023031022C3KC032033.map
factory_data/00D6-5BD8/iGPSPORT/Maps/AR10002303101ZA3JV01Y01Z.map
factory_data/00D6-5BD8/iGPSPORT/Maps/AR11002303101YL3TE03503H.map
factory_data/00D6-5BD8/iGPSPORT/Maps/AR12002303101XQ3O102Q034.map
factory_data/00D6-5BD8/iGPSPORT/Maps/AR13002303101X53R402M04C.map
factory_data/00D6-5BD8/iGPSPORT/Maps/AR140023031026B3MF01K01X.map
factory_data/00D6-5BD8/iGPSPORT/Maps/AR15002303101W

In [10]:
tags = {}
for f in files:
    header = parseHeader(f)

    for tag in header.wayTags:
        if not tag in tags.keys():
            tags[tag] = 1
        else:
            tags[tag] += 1

    # if b'natural=coastline' in header.wayTags:
    #     pprint.pp(f.name)
    #     pprint.pp(vars(header))
        
    pf = parseFilename(f.name)
    data.append({"file": f, "header": header, "parsedFilename": pf, "Name": f.name})
    
pprint.pp(tags)

{b'highway=cycleway': 423,
 b'highway=footway': 432,
 b'highway=living_street': 395,
 b'highway=path': 432,
 b'highway=pedestrian': 431,
 b'highway=primary': 430,
 b'highway=primary_link': 424,
 b'highway=residential': 432,
 b'highway=road': 293,
 b'highway=secondary': 432,
 b'highway=secondary_link': 423,
 b'highway=service': 432,
 b'highway=tertiary': 432,
 b'highway=tertiary_link': 426,
 b'highway=track': 432,
 b'highway=trunk': 421,
 b'highway=trunk_link': 421,
 b'highway=unclassified': 432,
 b'natural=coastline': 227,
 b'natural=water': 432}


In [11]:
names = [d["Name"] for d in data]
minLat = [d["header"].minLat for d in data]
minLon = [d["header"].minLon for d in data]
maxLat = [d["header"].maxLat for d in data]
maxLon = [d["header"].maxLon for d in data]

f1 = [d["parsedFilename"].field1 for d in data]
f2 = [d["parsedFilename"].field2 for d in data]
f3 = [d["parsedFilename"].z1 for d in data]
f4 = [d["parsedFilename"].z2 for d in data]

df = pd.DataFrame(
    {
        "name": names,
        "minLat": minLat,
        "minLon": minLon,
        "maxLat": maxLat,
        "maxLon": maxLon,
        "f1": f1,
        "f2": f2,
        "f3": f3,
        "f4": f4,
    }
)
df["minLat"] = df["minLat"]
df["minLon"] = df["minLon"]
df["maxLat"] = df["maxLat"]
df["maxLon"] = df["maxLon"]


In [12]:
traces = []

maxLon = -360
minLon = 360
maxLat = -360
minLat = 360

for index, row in df.iterrows():

    maxLon = row["maxLon"] if row["maxLon"] > maxLon else maxLon
    minLon = row["minLon"] if row["minLon"] < minLon else minLon
    maxLat = row["maxLat"] if row["maxLat"] > maxLat else maxLat
    minLat = row["minLat"] if row["minLat"] < minLat else minLat

    traces.append(
        go.Scattermap(
            mode="lines",
            fill="toself",
            name=row["name"],
            lon=[row["maxLon"], row["minLon"], row["minLon"], row["maxLon"]],
            lat=[row["maxLat"], row["maxLat"], row["minLat"], row["minLat"]],
            text=[row["name"], row["name"], row["name"], row["name"]],
        )
    )


traces.append(
    go.Scattermap(
        mode="lines",
        fill="toself",
        name="all",
        lon=[maxLon, minLon, minLon, maxLon],
        lat=[maxLat, maxLat, minLat, minLat],
        text=["all", "all", "all", "all"],
    )
)


fig = go.Figure(data=traces)

fig.update_layout(
    map={"style": "carto-darkmatter"},
    margin={"l": 0, "r": 0, "b": 0, "t": 0},
)

fig.write_html("docs/BoundingBoxes.html")